# Prelude

Contains the following:
1. Imports and defining helper functions
2. Scraping data from wikipedia and downloads it
3. Building the network either from
    - (A) Downloaded files
    - (B) Local pickle file (created from last time A was run)
4. Simple prelimenary data analysis of network

## 1. Imports

In [2]:
import shutil
import os
import networkx as nx
import pickle
from wiki_utils import getJsonResponse, findLinks, build_graph_from_files

# Set the directory to downloads
DOWNLOADS_DIR = "downloads"

## 2. Scraping data
Fetches philosopher data from their wikipedia pages and downloads the wikipedia pages as `{philosopher_name}.txt` in a `downloads/` directory.

>**NOTES**
> 1. This takes a while to run
> 2. It deletes all previous content in `downloads`
> 3. Downloads all pages but skips pages with *no content* or *redirects*.

In [3]:
wiki_links = ["List of philosophers (A–C)", "List of philosophers (D–H)", "List of philosophers (I–Q)", "List of philosophers (R–Z)"]
title_links = []

verbose = False # Debug output during loops
invalid_links = []  # Track titles that could not be saved
redirect_links = []  # Track titles that are redirects

# Delete and recreate the downloads directory
if os.path.exists(DOWNLOADS_DIR):
	shutil.rmtree(DOWNLOADS_DIR)  # Delete the directory and all its contents
os.makedirs(DOWNLOADS_DIR, exist_ok=True)  # Recreate the directory


for wiki_link in wiki_links:
  wiki_markup = getJsonResponse(wiki_link)
  title_links.extend(findLinks(wiki_markup))

# Remove irrelevant links if they exist
for unwanted in ["List_of_philosophers", "Philosopher", "Stanford_Encyclopedia_of_Philosophy", "Encyclopedia_of_Philosophy", "Routledge_Encyclopedia_of_Philosophy", "The_Cambridge_Dictionary_of_Philosophy", "The_Oxford_Companion_to_Philosophy"]:
	if unwanted in title_links:
		title_links.remove(unwanted)

# Writing to files (warning this takes a while)
for title_link in title_links:
	all_wikitext = getJsonResponse(title_link)
	if not all_wikitext:
		if verbose: print(f"Skipping '{title_link}' as it has no content.")
		invalid_links.append(title_link)  # Track invalid pages without modifying the list directly
		continue
  
  # Skip if the content starts with #REDIRECT
	if all_wikitext.strip().startswith("#REDIRECT"):
		if verbose: print(f"Skipping '{title_link}' as it is a redirect.")
		redirect_links.append(title_link)  # Track redirect pages
		continue
  
	filename = os.path.join(DOWNLOADS_DIR, f"{title_link}.txt")
	with open(filename, "w", encoding="utf-8") as file:
		file.write(all_wikitext) # save all the wikitext into one file

title_links = [link for link in title_links if link not in invalid_links + redirect_links]
print(f"Downloaded {len(title_links)} pages.")
print(f"Skipped {len(invalid_links)} pages with no content.")
print(f"Skipped {len(redirect_links)} redirect pages.")

Downloaded 1485 pages.
Skipped 32 pages with no content.
Skipped 249 redirect pages.


## 3. Building the network 

- (A) Create from scratch 
From `downloads/`directory (saves local pickle file for later)

In [4]:
S = build_graph_from_files(DOWNLOADS_DIR)
pickle.dump(S, open("graph.pkl", "wb")) # Saved as local version for later use for (B)

S_undirected = S.to_undirected()
# Identify self-loop edges
self_loops = list(nx.selfloop_edges(S_undirected))

# Print nodes with self-loops
print(f"Nodes with self-loops: {[edge[0] for edge in self_loops]}")

# Remove self-loops
S_undirected.remove_edges_from(self_loops)
pickle.dump(S_undirected, open("graph_undirected.pkl", "wb")) # Saved as local version for later use for (B)

Nodes with self-loops: ['Pierre_Teilhard_de_Chardin', 'Plato', 'Otto_Rühle', 'Shen_Buhai']


- (B) OR use local version 
From `pickle` file created last time you ran (A)

In [5]:
# load graph:
S = pickle.load(open("graph.pkl", "rb"))
S_undirected = pickle.load(open("graph_undirected.pkl", "rb"))

## 4. Prelimenary data analysis

In [8]:
print(f"Number of nodes in S: {S.number_of_nodes()}")
print(f"Number of edges in S: {S.number_of_edges()}, and in S_undirected: {S_undirected.number_of_edges()}")

# Calculating total data size
download_size = sum(os.path.getsize(os.path.join(DOWNLOADS_DIR, f)) for f in os.listdir(DOWNLOADS_DIR) if f.endswith(".txt"))
download_size_mb = download_size / (1024 * 1024)  # Convert bytes to MB
print(f"Size of downloaded data: {download_size_mb:.2f} MB")

Number of nodes in S: 1366
Number of edges in S: 10864, and in S_undirected: 9026
Size of downloaded data: 48.60 MB
